# VGG-16 Training 

In [13]:
import sys
!{sys.executable} -m pip install msgpack
!{sys.executable} -m pip install split-folders==0.2.0
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install sklearn

In [2]:
#Here we have all the packages needed to create our Neural Net
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D,AveragePooling2D,Input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import time
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
import itertools
import h5py
from keras.utils.io_utils import HDF5Matrix
from keras.models import Model
from keras.optimizers import Adam


Using TensorFlow backend.


## File Preprocessing (do once)

In [3]:
from zipfile import ZipFile
file_name='foodimages.zip'
zipp=ZipFile(file_name,mode='r')
zipp.extractall()

In [ ]:
import split_folders
split_folders.ratio('yum',output='output',ratio=(.8,.1,.1))

## Load Data (Start here)

In [3]:
train_image_dir='output/train'
val_image_dir='output/val'
test_image_dir='output/test'
image_size=(224,224)
batch_size=128

### Create Unaugmented Data Generator

In [4]:
datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = False,
fill_mode = "nearest",
zoom_range = 0,
width_shift_range = 0,
height_shift_range=0,
rotation_range=0)

train_generator = datagen.flow_from_directory(
train_image_dir,
target_size = (image_size[0], image_size[1]),
batch_size = batch_size, 
class_mode = "categorical")

val_generator = datagen.flow_from_directory(
val_image_dir,
target_size = (image_size[0], image_size[1]),
batch_size = batch_size, 
class_mode = "categorical")

test_generator = datagen.flow_from_directory(
test_image_dir,
target_size = (image_size[0], image_size[1]),
batch_size = batch_size, 
class_mode = "categorical")

Found 80800 images belonging to 101 classes.
Found 10100 images belonging to 101 classes.
Found 10100 images belonging to 101 classes.


### Augemented Data Generator

In [5]:
aug_gen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = False,
fill_mode = "nearest",
zoom_range = 0.25,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=30)

a_train_generator = aug_gen.flow_from_directory(
train_image_dir,
target_size = (image_size[0], image_size[1]),
batch_size = batch_size, 
class_mode = "categorical")

Found 80800 images belonging to 101 classes.


## Load VGG (Base) Model

In [2]:
from keras.applications.vgg16 import VGG16

In [19]:
# base_model=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(64,64,3),name='input_image'),classes=101)
base_model=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3),name='input_image'),classes=101)

In [20]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
# x=Flatten()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(1024,activation='relu')(x)
x=Dense(512,activation='relu')(x)
predictions=Dense(101,activation='softmax')(x)

In [21]:
vgg_model_1=Model(inputs=base_model.input,outputs=predictions)

In [23]:
vgg_model_1.compile(optimizer=Adam(lr=.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
vgg_model_1.fit_generator(train_generator,
               steps_per_epoch=632,
               epochs=1,
               verbose=1,
               validation_data=val_generator,
               validation_steps=batch_size,
               shuffle=True)

Epoch 1/1
632/632 [==============================] - 2401s 4s/step - loss: 4.1617 - acc: 0.0608 - val_loss: 3.3060 - val_acc: 0.1951


## Save model

In [6]:
from keras.models import model_from_json

In [7]:
def save_model(name_json, name_weights,model):
    model_json=model.to_json()
    with open(name_json,'w') as json_file:
        json_file.write(model_json)
    model.save_weights(name_weights)
    print('saved model to disk')

In [8]:
#load json and create model
def load_model(name_json,name_weights):
    json_file=open(name_json,'r')
    loaded_model_json=json_file.read()
    json_file.close()
    loaded_model=model_from_json(loaded_model_json)
    #load weights into new model
    loaded_model.load_weights(name_weights)
    print('Loaded model from disk')
    return loaded_model

## Training continues...

In [8]:
loaded_model.compile(optimizer=Adam(lr=.0001),loss='categorical_crossentropy',metrics=['accuracy'])
loaded_model.fit_generator(train_generator,
               steps_per_epoch=632,
               epochs=2,
               verbose=1,
               validation_data=val_generator,
               validation_steps=batch_size,
               shuffle=True)

Epoch 1/2
632/632 [==============================] - 2608s 4s/step - loss: 2.6654 - acc: 0.3272 - val_loss: 2.1546 - val_acc: 0.4447
Epoch 2/2
632/632 [==============================] - 2513s 4s/step - loss: 1.9260 - acc: 0.4974 - val_loss: 1.7630 - val_acc: 0.5417


In [12]:
save_model('vgg_model_1_1.json','vgg_model_1_1_weights.h5',loaded_model)

saved model to disk


In [12]:
vgg_model_1_1=load_model('vgg_model_1_1.json','vgg_model_1_1_weights.h5')

Instructions for updating:
Colocations handled automatically by placer.
Loaded model from disk


In [14]:
vgg_model_1_1.compile(optimizer=Adam(lr=.0001),loss='categorical_crossentropy',metrics=['accuracy'])
vgg_model_1_1.fit_generator(train_generator,
               steps_per_epoch=632,
               epochs=5,
               verbose=1,
               validation_data=val_generator,
               validation_steps=batch_size,
               shuffle=True)

Epoch 1/5
632/632 [==============================] - 2134s 3s/step - loss: 1.5294 - acc: 0.5936 - val_loss: 1.5565 - val_acc: 0.5924
Epoch 2/5
632/632 [==============================] - 624s 988ms/step - loss: 1.2932 - acc: 0.6516 - val_loss: 1.4588 - val_acc: 0.6308
Epoch 3/5
632/632 [==============================] - 622s 984ms/step - loss: 1.1085 - acc: 0.6960 - val_loss: 1.3376 - val_acc: 0.6533
Epoch 4/5
632/632 [==============================] - 621s 983ms/step - loss: 0.9603 - acc: 0.7321 - val_loss: 1.3129 - val_acc: 0.6725
Epoch 5/5
632/632 [==============================] - 622s 984ms/step - loss: 0.8156 - acc: 0.7701 - val_loss: 1.2669 - val_acc: 0.6829


In [15]:
save_model('vgg_model_1_1.json','vgg_model_1_1_weights.h5',vgg_model_1_1)

saved model to disk


In [16]:
vgg_model_1_1.fit_generator(train_generator,
               steps_per_epoch=632,
               epochs=10,
               verbose=1,
               validation_data=val_generator,
               validation_steps=batch_size,
               shuffle=True)

Epoch 1/10
632/632 [==============================] - 625s 988ms/step - loss: 0.6906 - acc: 0.8013 - val_loss: 1.3028 - val_acc: 0.6877
Epoch 2/10
632/632 [==============================] - 624s 988ms/step - loss: 0.5810 - acc: 0.8301 - val_loss: 1.4755 - val_acc: 0.6666
Epoch 3/10
632/632 [==============================] - 623s 986ms/step - loss: 0.4923 - acc: 0.8544 - val_loss: 1.4129 - val_acc: 0.6728
Epoch 4/10
632/632 [==============================] - 624s 988ms/step - loss: 0.4087 - acc: 0.8763 - val_loss: 1.4170 - val_acc: 0.6887
Epoch 5/10
632/632 [==============================] - 624s 987ms/step - loss: 0.3338 - acc: 0.8983 - val_loss: 1.6071 - val_acc: 0.6854
Epoch 6/10
632/632 [==============================] - 624s 987ms/step - loss: 0.2819 - acc: 0.9116 - val_loss: 1.5458 - val_acc: 0.6950
Epoch 7/10
632/632 [==============================] - 625s 989ms/step - loss: 0.2417 - acc: 0.9244 - val_loss: 1.6669 - val_acc: 0.6880
Epoch 8/10
632/632 [============================

In [17]:
save_model('vgg_model_1_2.json','vgg_model_1_2_weights.h5',vgg_model_1_1)

saved model to disk


In [18]:
vgg_model_1_1.fit_generator(train_generator,
               steps_per_epoch=632,
               epochs=10,
               verbose=1,
               validation_data=val_generator,
               validation_steps=batch_size,
               shuffle=True)

Epoch 1/10
632/632 [==============================] - 625s 988ms/step - loss: 0.1464 - acc: 0.9533 - val_loss: 1.7762 - val_acc: 0.6853
Epoch 2/10
632/632 [==============================] - 621s 983ms/step - loss: 0.1367 - acc: 0.9567 - val_loss: 1.8909 - val_acc: 0.6884
Epoch 3/10
632/632 [==============================] - 623s 986ms/step - loss: 0.1345 - acc: 0.9569 - val_loss: 1.7496 - val_acc: 0.6862
Epoch 4/10
632/632 [==============================] - 623s 987ms/step - loss: 0.1191 - acc: 0.9617 - val_loss: 1.9071 - val_acc: 0.6884
Epoch 5/10
632/632 [==============================] - 624s 987ms/step - loss: 0.1105 - acc: 0.9648 - val_loss: 2.0331 - val_acc: 0.6800
Epoch 6/10
632/632 [==============================] - 623s 986ms/step - loss: 0.1017 - acc: 0.9672 - val_loss: 1.7711 - val_acc: 0.6940
Epoch 7/10
632/632 [==============================] - 624s 988ms/step - loss: 0.1008 - acc: 0.9683 - val_loss: 1.8762 - val_acc: 0.6895
Epoch 8/10
632/632 [============================

In [19]:
save_model('vgg_model_1_3.json','vgg_model_1_3_weights.h5',vgg_model_1_1)

saved model to disk


## Tweaking by adding Dropout Layers

In [1]:
vgg_model_2=load_model('vgg_models/vgg_model_1_3.json','vgg_models/vgg_model_1_3_weights.h5')

In [9]:
vgg_model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [12]:
fc1=vgg_model_2.layers[-4]
fc2=vgg_model_2.layers[-3]
fc3=vgg_model_2.layers[-2]
predictions=vgg_model_2.layers[-1]

In [13]:
dropout1=Dropout(.5)
dropout2=Dropout(.5)

In [14]:
#Reconnect the layers
x=dropout1(fc1.output)
x=fc2(x)
x=dropout2(x)
x=fc3(x)
predictors=predictions(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Training with dropout layers

In [15]:
vgg_model_2=Model(input=vgg_model_2.input,output=predictors)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [16]:
vgg_model_2.compile(optimizer=Adam(lr=.0001),loss='categorical_crossentropy',metrics=['accuracy'])
vgg_model_2.fit_generator(train_generator,
               steps_per_epoch=632,
               epochs=1,
               verbose=1,
               validation_data=val_generator,
               validation_steps=batch_size,
               shuffle=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
632/632 [==============================] - 2440s 4s/step - loss: 0.3107 - acc: 0.9051 - val_loss: 1.6107 - val_acc: 0.6995


In [18]:
save_model('vgg_models/vgg_model_2.json','vgg_models/vgg_model_2_weights.h5',vgg_model_2)

saved model to disk


In [27]:
vgg_model_2.fit_generator(train_generator,
               steps_per_epoch=632,
               epochs=6,
               verbose=1,
               validation_data=val_generator,
               validation_steps=batch_size,
               shuffle=True)

Epoch 1/6
632/632 [==============================] - 622s 985ms/step - loss: 0.2215 - acc: 0.9309 - val_loss: 1.6566 - val_acc: 0.6982
Epoch 2/6
632/632 [==============================] - 622s 985ms/step - loss: 0.1997 - acc: 0.9389 - val_loss: 1.6598 - val_acc: 0.6921
Epoch 3/6
632/632 [==============================] - 620s 981ms/step - loss: 0.1791 - acc: 0.9462 - val_loss: 1.7189 - val_acc: 0.6970
Epoch 4/6
632/632 [==============================] - 621s 982ms/step - loss: 0.1609 - acc: 0.9510 - val_loss: 1.8109 - val_acc: 0.6858
Epoch 5/6
632/632 [==============================] - 623s 985ms/step - loss: 0.1454 - acc: 0.9559 - val_loss: 1.7514 - val_acc: 0.6972
Epoch 6/6
632/632 [==============================] - 622s 984ms/step - loss: 0.1378 - acc: 0.9580 - val_loss: 1.8857 - val_acc: 0.6734


## Train on Augmented Data

In [9]:
vgg_model_3=load_model('vgg_models/vgg_model_2.json','vgg_models/vgg_model_2_weights.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


In [10]:
vgg_model_3.compile(optimizer=Adam(lr=.0001),loss='categorical_crossentropy',metrics=['accuracy'])
vgg_model_3.fit_generator(a_train_generator,
               steps_per_epoch=632,
               epochs=5,
               verbose=1,
               validation_data=val_generator,
               validation_steps=batch_size,
               shuffle=True)
save_model('vgg_models/vgg_model_3.json','vgg_models/vgg_model_3_weights.h5',vgg_model_3)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
632/632 [==============================] - 3130s 5s/step - loss: 0.9154 - acc: 0.7518 - val_loss: 1.3649 - val_acc: 0.6885
Epoch 2/5
632/632 [==============================] - 1049s 2s/step - loss: 0.8199 - acc: 0.7765 - val_loss: 1.2612 - val_acc: 0.7081
Epoch 3/5
632/632 [==============================] - 1053s 2s/step - loss: 0.7569 - acc: 0.7899 - val_loss: 1.2458 - val_acc: 0.7174
Epoch 4/5
632/632 [==============================] - 1046s 2s/step - loss: 0.7022 - acc: 0.8039 - val_loss: 1.2595 - val_acc: 0.7075
Epoch 5/5
632/632 [==============================] - 1041s 2s/step - loss: 0.6647 - acc: 0.8141 - val_loss: 1.2781 - val_acc: 0.7075
saved model to disk


## Create dictionary to get from keras prediction to label name. (Only need to do once!)

In [19]:
label_map=(train_generator.class_indices)

In [60]:
my_dictionary={y:x for x,y in label_map.items()}

In [64]:
my_dictionary[0]

'apple_pie'

In [69]:
import pickle

In [70]:
output=open('label_dictionary.pkl','wb')
pickle.dump(my_dictionary,output)
output.close()

In [71]:
pkl_file=open('label_dictionary.pkl','rb')
try_again=pickle.load(pkl_file)
pkl_file.close()

In [73]:
try_again[0]

'apple_pie'

## Create dictionary for food to calories. (Only need to do once!)

In [26]:
file=open('food_names.txt','w')
for food in label_map:
    file.write(food)
    file.write(' '+'\n')
#     file.write('/n')
file.close()